In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import pandas as pd
import numpy as np
np.random.seed(132)
from functools import lru_cache

import sys

CODE_PATH = '../code'

sys.path.append(CODE_PATH)
import functions

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.datasets import load_iris, load_boston, load_breast_cancer, load_wine, load_digits
from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

from tqdm import tqdm

%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Берем выборку

In [3]:
# LETTER
letter = pd.read_csv('../data/letter.csv', header=None)
X = letter.drop(0, 1).values
target = pd.factorize(letter[0])[0]

In [ ]:
# POKER
# https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass.html
from sklearn.datasets import load_svmlight_file
data = load_svmlight_file('../data/poker')
X, target = data[0].toarray(), data[1].astype(int)
X, target = X[target <= 7], target[target <= 7]
pd.Series(target).value_counts()

In [ ]:
# Sensorless
# https://archive.ics.uci.edu/ml/datasets/dataset+for+sensorless+drive+diagnosis
from sklearn.datasets import load_svmlight_file
data = load_svmlight_file('../data/Sensorless.scale')
X, target = data[0].toarray(), data[1].astype(int)-1
X, _, target, _ = train_test_split(X, target, train_size=0.2, random_state=42, stratify=target)

pd.Series(target).value_counts()

In [ ]:
# 20NEWS
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups

twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
cv = TfidfVectorizer(max_features=100)
cv = CountVectorizer(max_features=100)

X = cv.fit_transform(twenty_train.data).toarray()
target = twenty_train.target

In [34]:
# DIGITS
dataset = load_digits()
df = pd.DataFrame(dataset['data'])
target = dataset['target']

X = df.values

In [18]:
# MODEL
df = pd.read_csv('../data/model_16_16.csv')
target = df['target'].astype(int).values
del df['target']

X = df.values

In [35]:
# TRAIN/TEST/VALID SPLIT
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42, stratify=target)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify=y_train)
print(X_train.shape, X_test.shape, X_val.shape)

(1005, 64) (360, 64) (432, 64)


In [94]:
BaseClassifier = LinearSVC

# N случайных дихотомий (без отбора)

In [44]:
attempts_data = []
N_attempts = 10
l = np.unique(target).size
N = 30 # кол-во дихотомий
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_random_dichs(l, N)
    print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, BaseClassifier)
    for score_type in wtypes:
        weight_type = None
        preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)
#             print(score_type, weight_type, accuracy_score(preds, y_val))

Training dich classifiers:   7%|▋         | 2/30 [00:00<00:01, 18.19it/s]

Code Matrix shape == (10,30)



Training dich classifiers:  10%|█         | 3/30 [00:00<00:01, 24.52it/s]

Code Matrix shape == (10,30)



Training dich classifiers:  10%|█         | 3/30 [00:00<00:01, 26.03it/s]

...


Training dich classifiers:   7%|▋         | 2/30 [00:00<00:01, 15.26it/s]

Code Matrix shape == (10,30)



Training dich classifiers:  10%|█         | 3/30 [00:00<00:00, 27.53it/s]

Code Matrix shape == (10,30)



100%|██████████| 10/10 [00:29<00:00,  2.96s/it]30 [00:01<00:00, 22.34it/s]


In [45]:
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

,sNone,saccuracy,sf1,sconfusion_list,snum_real_dich
count,10.000000,10.000000,10.000000,10.000000,10.0
mean,0.886574,0.896296,0.892593,0.903704,30.0
std,0.026189,0.026162,0.027782,0.018844,0.0
min,0.835648,0.840278,0.831019,0.868056,30.0
25%,0.877894,0.891204,0.885417,0.895255,30.0
50%,0.885417,0.898148,0.891204,0.905093,30.0
75%,0.901042,0.915509,0.916088,0.909144,30.0
max,0.923611,0.930556,0.923611,0.939815,30.0


In [52]:
# делаем общее многозначное
attempts_data = []
for N in np.linspace(10, 200, 20, dtype=int): # кол-во дихотомий
    N_attempts = 10
    l = np.unique(target).size
    wtypes = [None, 'accuracy', 'f1', 'confusion_list']
    for i in tqdm(range(N_attempts)):
        accs = []
        code_matrix = functions.make_random_dichs(l, N)
        print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
        dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                                 X_test, y_test, BaseClassifier)
        for score_type in wtypes:
            weight_type = None
            preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
            acc = accuracy_score(preds, y_val)
            accs.append(acc)
        accs.append(num_real_dich)
        attempts_data.append(accs)
    #             print(score_type, weight_type, accuracy_score(preds, y_val))


Training dich classifiers:  10%|█         | 1/10 [00:00<00:00,  9.70it/s]

Code Matrix shape == (10,10)



Training dich classifiers:  40%|████      | 4/10 [00:00<00:00, 34.33it/s]

Code Matrix shape == (10,10)



Training dich classifiers:  40%|████      | 4/10 [00:00<00:00, 34.99it/s]

...


Training dich classifiers:   2%|▏         | 3/200 [00:00<00:09, 21.30it/s]

Code Matrix shape == (10,200)



Training dich classifiers:   2%|▏         | 3/200 [00:00<00:10, 18.09it/s]

Code Matrix shape == (10,200)



100%|██████████| 10/10 [03:06<00:00, 18.61s/it]/200 [00:10<00:00, 18.34it/s]


In [65]:
cols = ['ecoc', 'accuracy', 'f1', 'confusion_list', 'num_real_dich']
df_attempts = pd.DataFrame(attempts_data, columns=cols)

gb = df_attempts.groupby('num_real_dich').agg(['mean', 'std'])
gb.columns = ['_'.join(col).strip() for col in gb.columns.values]
gb = gb.reset_index()
gb

,num_real_dich,ecoc_mean,ecoc_std,accuracy_mean,accuracy_std,f1_mean,f1_std,confusion_list_mean,confusion_list_std
0,10,0.756481,0.026613,0.779398,0.024741,0.777083,0.027971,0.774074,0.033442
1,20,0.848611,0.025479,0.857639,0.020567,0.861111,0.019306,0.860648,0.015112
2,30,0.887731,0.018750,0.893981,0.014673,0.892824,0.016261,0.897222,0.011032
3,40,0.910648,0.011914,0.914352,0.010912,0.915509,0.012358,0.917361,0.009578
4,50,0.912269,0.017096,0.919444,0.012902,0.918519,0.013221,0.921296,0.016585
5,60,0.917361,0.016730,0.925000,0.015593,0.925231,0.015853,0.924074,0.016067
6,70,0.919444,0.015501,0.925000,0.015207,0.924306,0.015626,0.930093,0.014261
7,80,0.921296,0.016513,0.928472,0.013160,0.931481,0.011032,0.933102,0.010937
8,90,0.930093,0.014994,0.935417,0.009479,0.937731,0.009665,0.938657,0.010816
9,100,0.927315,0.010535,0.932639,0.010661,0.932176,0.009640,0.940509,0.011906


# Отбор зазором

In [41]:
attempts_data = []
N_attempts = 10
l = np.unique(target).size
N = 100 # кол-во дихотомий
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_random_dichs(l, N)
#     print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, BaseClassifier)
    for score_type in wtypes:
        preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, 
                                      code_matrix, score_type, weight_type=-1)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)

Training dich classifiers:  38%|███▊      | 38/100 [00:00<00:00, 125.13it/s]


100%|██████████| 10/10 [00:59<00:00,  5.90s/it]/100 [00:00<00:00, 113.52it/s]


In [42]:
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

,sNone,saccuracy,sf1,sconfusion_list,snum_real_dich
count,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.973843,0.975463,0.976389,0.949769,43.000000
std,0.005675,0.005897,0.004856,0.010915,2.054805
min,0.967593,0.967593,0.972222,0.937500,40.000000
25%,0.970486,0.972222,0.972222,0.942708,42.000000
50%,0.972222,0.972222,0.974537,0.946759,42.500000
75%,0.973958,0.978588,0.979167,0.954282,44.500000
max,0.986111,0.986111,0.986111,0.972222,46.000000


In [95]:
# делаем общее многозначное
attempts_data = []
for N in np.linspace(20, 200, 10, dtype=int): # кол-во дихотомий
    N_attempts = 10
    l = np.unique(target).size
    wtypes = [None, 'accuracy', 'f1', 'confusion_list']
    for i in tqdm(range(N_attempts)):
        accs = []
        code_matrix = functions.make_random_dichs(l, N)
    #     print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
        dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                                 X_test, y_test, BaseClassifier)
        for score_type in wtypes:
            preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, 
                                          code_matrix, score_type, weight_type=-1)
            acc = accuracy_score(preds, y_val)
            accs.append(acc)
        accs.append(num_real_dich)
        attempts_data.append(accs)

Adding dich: 100%|██████████| 20/20 [00:00<00:00, 22174.49it/s]


Training dich classifiers:  72%|███████▎  | 29/40 [00:01<00:00, 16.86it/s]


Training dich classifiers:  18%|█▊        | 11/60 [00:00<00:02, 16.60it/s]


Training dich classifiers:  87%|████████▋ | 52/60 [00:02<00:00, 17.90it/s]


Training dich classifiers:  67%|██████▋   | 40/60 [00:02<00:01, 15.91it/s]


...

Training dich classifiers:  78%|███████▊  | 156/200 [00:08<00:02, 18.27it/s]


Training dich classifiers:   4%|▍         | 9/200 [00:00<00:07, 25.90it/s]


Training dich classifiers:  28%|██▊       | 57/200 [00:03<00:08, 17.76it/s]


Training dich classifiers:  60%|██████    | 120/200 [00:06<00:04, 18.55it/s]


100%|██████████| 10/10 [03:13<00:00, 19.37s/it]/200 [00:10<00:00, 18.35it/s]


In [79]:
cols = ['ecoc', 'accuracy', 'f1', 'confusion_list', 'num_real_dich']
df_attempts = pd.DataFrame(attempts_data, columns=cols)
df_attempts['gb_index'] = (df_attempts.index / 10).astype(int)

gb = df_attempts.groupby('gb_index').agg(['mean', 'std'])
gb.columns = ['_'.join(col).strip() for col in gb.columns.values]
gb = gb.reset_index()
gb

,gb_index,ecoc_mean,ecoc_std,accuracy_mean,accuracy_std,f1_mean,f1_std,confusion_list_mean,confusion_list_std,num_real_dich_mean,num_real_dich_std
0,0,0.928704,0.011894,0.929398,0.010706,0.927315,0.012013,0.854167,0.053112,12.5,3.535534
1,1,0.958796,0.007385,0.959954,0.008456,0.959722,0.008878,0.919676,0.018064,23.1,1.911951
2,2,0.965741,0.004604,0.966435,0.006292,0.965741,0.005324,0.934722,0.010846,30.7,1.888562
3,3,0.969907,0.004499,0.969907,0.004499,0.969213,0.004233,0.945370,0.013058,39.0,1.763834
4,4,0.974769,0.005054,0.975000,0.006437,0.974769,0.006854,0.951852,0.009929,43.1,1.911951
5,5,0.973380,0.003819,0.974306,0.004002,0.975231,0.003942,0.953704,0.007238,45.8,2.898275
6,6,0.977546,0.003942,0.978472,0.004090,0.978472,0.003627,0.958333,0.008166,46.7,1.888562
7,7,0.975926,0.004904,0.976620,0.004687,0.976157,0.005007,0.955787,0.010991,48.7,2.540779
8,8,0.974769,0.004687,0.975231,0.005124,0.975231,0.005007,0.952546,0.006923,49.3,2.710064
9,9,0.977546,0.005351,0.978472,0.004763,0.978704,0.004604,0.955093,0.008034,51.0,4.988877


# Дихотомии где остались лучшие по критерию

In [ ]:
attempts_data = []
N_attempts = 10
l = np.unique(target).size
N = 300 # кол-во дихотомий
num_real_dich = 100 # кол-во дихотомий которые оставляем
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_random_dichs(l, N)
    print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, BaseClassifier)
    
    metric_map = np.array([d['f1'] for d in dich_classifiers])
    top_dich = np.sort(np.argsort(metric_map)[-num_real_dich:])
    dich_classifiers = [d for i,d in enumerate(dich_classifiers) if i in top_dich]
    code_matrix = code_matrix.T[top_dich].T
    
    for score_type in wtypes:
        weight_type = None
        preds, _ = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)
#             print(score_type, weight_type, accuracy_score(preds, y_val))

In [ ]:
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

In [100]:
%%time
attempts_data = []
N = 400
l = np.unique(target).size
for num_real_dich in np.linspace(20, 140, 13, dtype=int): # кол-во дихотомий
    N_attempts = 10
    wtypes = [None, 'accuracy', 'f1', 'confusion_list']
    for i in tqdm(range(N_attempts)):
        accs = []
        code_matrix = functions.make_random_dichs(l, N)
        print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
        dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                                 X_test, y_test, BaseClassifier)

        metric_map = np.array([d['f1'] for d in dich_classifiers])
        top_dich = np.sort(np.argsort(metric_map)[-num_real_dich:])
        dich_classifiers = [d for i,d in enumerate(dich_classifiers) if i in top_dich]
        code_matrix = code_matrix.T[top_dich].T

        for score_type in wtypes:
            weight_type = None
            preds, _ = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
            acc = accuracy_score(preds, y_val)
            accs.append(acc)
        accs.append(num_real_dich)
        attempts_data.append(accs)

Training dich classifiers:   0%|          | 2/400 [00:00<00:36, 10.88it/s]

Code Matrix shape == (10,400)



Training dich classifiers:  62%|██████▏   | 247/400 [00:14<00:08, 17.29it/s]


Training dich classifiers:   1%|          | 3/400 [00:00<00:18, 20.90it/s]

Code Matrix shape == (10,400)


...

Training dich classifiers:   1%|          | 3/400 [00:00<00:19, 20.05it/s]

Code Matrix shape == (10,400)



Training dich classifiers:  68%|██████▊   | 270/400 [00:14<00:06, 18.84it/s]


100%|██████████| 10/10 [04:36<00:00, 27.68s/it]/400 [00:21<00:00, 18.77it/s]

CPU times: user 3h 13min 38s, sys: 5h 53min 57s, total: 9h 7min 36s
Wall time: 54min 40s


In [101]:
cols = ['ecoc', 'accuracy', 'f1', 'confusion_list', 'num_real_dich']
df_attempts = pd.DataFrame(attempts_data, columns=cols)

gb = df_attempts.groupby('num_real_dich').agg(['mean', 'std'])
gb.columns = ['_'.join(col).strip() for col in gb.columns.values]
gb = gb.reset_index()
gb

,num_real_dich,ecoc_mean,ecoc_std,accuracy_mean,accuracy_std,f1_mean,f1_std,confusion_list_mean,confusion_list_std
0,20,0.839815,0.026162,0.848611,0.025197,0.844907,0.026528,0.843519,0.028460
1,30,0.905556,0.026366,0.906944,0.032034,0.909954,0.026939,0.909259,0.026343
2,40,0.931944,0.010591,0.932407,0.010398,0.932176,0.011806,0.929861,0.011290
3,50,0.934491,0.010969,0.936111,0.012013,0.934722,0.012141,0.935648,0.008078
4,60,0.947454,0.007873,0.947222,0.008438,0.947685,0.006832,0.946065,0.007564
5,70,0.946759,0.005233,0.947454,0.006819,0.947222,0.005324,0.946065,0.005569
6,80,0.952315,0.007576,0.953009,0.006906,0.953241,0.007304,0.954861,0.006292
7,90,0.951852,0.007385,0.952778,0.006565,0.952315,0.007576,0.954630,0.006382
8,100,0.953241,0.006709,0.953241,0.006620,0.953009,0.006552,0.954861,0.004532
9,110,0.953009,0.005007,0.953241,0.004856,0.953241,0.004977,0.955324,0.005675


# Локальный метод оптимизации

In [111]:
def score_function(cur_dich):
    # cluster
    if cur_dich.max() == cur_dich.min(): #trivial dich
        return -inf
    target = np.array([cur_dich[i] for i in y_train])
    return -functions.cluster_score(X_train, target, score_type='trace_w')

In [123]:
def score_function(cur_dich):
    # accuracy
    X = X_train
    y = np.array([cur_dich[i] for i in y_train])
    if y.max() == y.min(): #trivial dich
        return 0
    clf = LogisticRegression()
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    y_true = np.array([cur_dich[i] for i in y_test])
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

In [105]:
def score_function(cur_dich):
    # f1
    X = X_train
    y = np.array([cur_dich[i] for i in y_train])
    if y.max() == y.min(): #trivial dich
        return 0
    clf = LogisticRegression()
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    y_true = np.array([cur_dich[i] for i in y_test])
    accuracy = f1_score(y_true, y_pred)
    return accuracy

In [112]:
attempts_data = []
N_attempts = 10
l = np.unique(target).size
N = 30 # кол-во дихотомий
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    
    code_matrix = functions.make_code_matrix_local(l, N, score_function, 0)
    while code_matrix.sum(axis=0).max() == l or code_matrix.sum(axis=0).min() == 0:
        code_matrix = functions.make_code_matrix_local(l, N, score_function, 0)
    print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, BaseClassifier)
    for score_type in wtypes:
        weight_type = None
        preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)

100%|██████████| 30/30 [00:02<00:00, 14.47it/s]

Training dich classifiers:  30%|███       | 9/30 [00:00<00:00, 89.09it/s]

Code Matrix shape == (10,30)



100%|██████████| 30/30 [00:02<00:00, 14.51it/s]

Training dich classifiers:  33%|███▎      | 10/30 [00:00<00:00, 86.61it/s]

Code Matrix shape == (10,30)



100%|██████████| 30/30 [00:02<00:00, 13.83it/s]

100%|██████████| 30/30 [00:02<00:00, 13.96it/s]

100%|██████████| 30/30 [00:01<00:00, 15.06it/s]

100%|██████████| 30/30 [00:01<00:00, 16.76it/s]

100%|██████████| 30/30 [00:02<00:00, 14.71it/s]

 97%|█████████▋| 29/30 [00:02<00:00, 14.11it/s]


...


100%|██████████| 30/30 [00:02<00:00, 14.05it/s]

100%|██████████| 30/30 [00:02<00:00, 13.93it/s]

Training dich classifiers:  27%|██▋       | 8/30 [00:00<00:00, 72.51it/s]

Code Matrix shape == (10,30)



100%|██████████| 30/30 [00:02<00:00, 14.97it/s]

Training dich classifiers:  27%|██▋       | 8/30 [00:00<00:00, 71.82it/s]

Code Matrix shape == (10,30)



100%|██████████| 10/10 [01:31<00:00,  9.17s/it]30 [00:00<00:00, 66.94it/s]


In [113]:
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

,sNone,saccuracy,sf1,sconfusion_list,snum_real_dich
count,10.000000,10.000000,10.000000,10.000000,10.0
mean,0.524537,0.521296,0.520833,0.536343,30.0
std,0.049373,0.056668,0.057711,0.051750,0.0
min,0.388889,0.368056,0.361111,0.391204,30.0
25%,0.524306,0.524884,0.523727,0.542824,30.0
50%,0.538194,0.535880,0.533565,0.553241,30.0
75%,0.545718,0.548611,0.546296,0.557292,30.0
max,0.560185,0.562500,0.562500,0.564815,30.0


In [120]:
np.linspace(20, 55, 8, dtype=int)

array([20, 25, 30, 35, 40, 45, 50, 55])

In [124]:
attempts_data = []
N_attempts = 10
l = np.unique(target).size
wtypes = [None, 'accuracy', 'f1', 'confusion_list']


subset = np.linspace(20, 200, 10, dtype=int) # кол-во дихотомий
subset = np.linspace(20, 55, 8, dtype=int)
for N in subset: 
    for i in tqdm(range(N_attempts)):
        accs = []
        code_matrix = functions.make_code_matrix_local(l, N, score_function, 0)
        while code_matrix.sum(axis=0).max() == l or code_matrix.sum(axis=0).min() == 0:
            code_matrix = functions.make_code_matrix_local(l, N, score_function, 0)
        print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
        dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                                 X_test, y_test, BaseClassifier)
        for score_type in wtypes:
            weight_type = None
            preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
            acc = accuracy_score(preds, y_val)
            accs.append(acc)
        accs.append(num_real_dich)
        attempts_data.append(accs)

100%|██████████| 20/20 [00:12<00:00,  1.57it/s]

Training dich classifiers:  45%|████▌     | 9/20 [00:00<00:00, 86.55it/s]

Code Matrix shape == (10,20)



100%|██████████| 20/20 [00:12<00:00,  1.56it/s]

Training dich classifiers:  60%|██████    | 12/20 [00:00<00:00, 114.66it/s]

Code Matrix shape == (10,20)



100%|██████████| 20/20 [00:13<00:00,  1.50it/s]

Training dich classifiers:  50%|█████     | 10/20 [00:00<00:00, 96.78it/s]

...


100%|██████████| 55/55 [00:30<00:00,  1.78it/s]

Training dich classifiers:  18%|█▊        | 10/55 [00:00<00:00, 92.19it/s]

Code Matrix shape == (10,55)



100%|██████████| 55/55 [00:33<00:00,  1.64it/s]

Training dich classifiers:  11%|█         | 6/55 [00:00<00:00, 57.33it/s]

Code Matrix shape == (10,55)



100%|██████████| 10/10 [05:36<00:00, 33.61s/it]55 [00:01<00:00, 52.99it/s]


In [125]:
cols = ['ecoc', 'accuracy', 'f1', 'confusion_list', 'num_real_dich']
df_attempts = pd.DataFrame(attempts_data, columns=cols)

gb = df_attempts.groupby('num_real_dich').agg(['mean', 'std'])
gb.columns = ['_'.join(col).strip() for col in gb.columns.values]
gb = gb.reset_index()
gb

,num_real_dich,ecoc_mean,ecoc_std,accuracy_mean,accuracy_std,f1_mean,f1_std,confusion_list_mean,confusion_list_std
0,20,0.899306,0.028082,0.902315,0.033912,0.906250,0.030383,0.908102,0.027736
1,25,0.923380,0.018693,0.927778,0.015386,0.926157,0.014495,0.928472,0.016385
2,30,0.925694,0.023415,0.929167,0.025479,0.930093,0.025259,0.928472,0.030308
3,35,0.931944,0.029366,0.931019,0.030976,0.932176,0.029241,0.932176,0.025499
4,40,0.946759,0.006363,0.947917,0.006923,0.946065,0.007076,0.945833,0.006382
5,45,0.946528,0.007978,0.952083,0.007797,0.950694,0.006906,0.951852,0.004604
6,50,0.949769,0.007405,0.951852,0.006970,0.950231,0.006197,0.949769,0.008170
7,55,0.949074,0.007793,0.950694,0.007948,0.949306,0.008962,0.951157,0.006588
